In [ ]:
# Description of data
tennisMain = "../programs/tennis.py"
refgtf     = "../data/dm6.gtf"

First we read the gtf file, representing isoforms as binary vectors

In [ ]:
# read file and get chains
import sys
import os

# get the path to tennis/src dir
tennisSrc = tennisMain
while os.path.islink(tennisSrc):
    newTarget = os.readlink(tennisSrc)
    if not os.path.isabs(newTarget):
        newTarget = os.path.join(os.path.dirname(os.path.abspath(tennisSrc)), newTarget)
    tennisSrc = newTarget
tennisSrcDir = os.path.dirname(tennisSrc)
sys.path.insert(0, tennisSrcDir)


from tennis import Transcriptom

!mkdir removal_retrival 
save_basename = 'removal_retrival/Ref_removal'
tsm = Transcriptom(refgtf,  f'{save_basename + "tmp"}.stats', f'{save_basename + "tmp"}.pred.gtf')

multi_iso_gene = 0
for i in tsm.exonMatrixCollection:
    if len(i) >= 2:
        multi_iso_gene += 1


matrix = tsm.get_chain_matrix('pexon_chain', 'tsstes_level')

Then we removed 1 isoform from each multi-isoform transcript group if (1) this removal does not remove the shortest transcript; (2) this removal does not reduce total number of exons. The purpose of those two conditions are that the removed isoforms should be theoretically able to be retrieved. 

In [ ]:
# remove One transcript from gene
# if gene has multiple transcripts and
# 1: this removal should not remove the shortest transcript
# 2: this removal should not reduce total number of exons
import random
from collections import Counter
from collections import defaultdict

considered_gene_count = 0
# the sum of removed + ramaining is less than total because some genes don't satisfy the conditions
removed_transcripts = []
remaining_transcripts = []
assert len(matrix) == len(tsm.matrix_gids)
for i in range(len(matrix)):
    group_id = tsm.matrix_gids[i]
    group_isoforms = matrix[i]

    if len(group_isoforms) <= 2: continue

    # count exons and get shortest transcript length
    shortest_isoform_length = len(group_isoforms[0])
    exon_counter = Counter()
    for isoform in group_isoforms:
        if len(isoform) <= 2: # must be multi-exon transcripts
            continue
        if len(isoform) < shortest_isoform_length:
            shortest_isoform_length = len(isoform)
        exon_counter.update(isoform)

    # count exons and get shortest transcript length
    random.shuffle(group_isoforms)
    for i, isoform in enumerate(group_isoforms):
        # condition 1
        if len(isoform) == shortest_isoform_length:
            continue

        # condition 2
        is_exon_num_reduced = False
        for exon in isoform:
            if exon_counter[exon] <= 1:
                is_exon_num_reduced = True
                break
        if is_exon_num_reduced:
            continue

        prev_iso_num = len(group_isoforms)
        removed_transcripts.append((group_id, isoform))
        group_isoforms.remove(isoform)
        remaining_transcripts.append((group_id, group_isoforms))
        assert len(group_isoforms) + 1 == prev_iso_num
        considered_gene_count += 1
        break

print(f'considered {considered_gene_count} transcript groups')
for i, j in zip(removed_transcripts[:10], remaining_transcripts[:10]):
    print("removed:", i)
    print(f"remaining ({len(j)}):", j)

In [ ]:
# save the removed and remaining transcripts
import os
remaining_file = save_basename + ".remaining.gtf"
if os.path.exists(remaining_file):
    print(remaining_file, "exists. Removing it")
    os.remove(remaining_file)
for gid, chains in remaining_transcripts:
    info = dict()
    tsm.save_novel_gene_in_gtf(chains, info, gid, "pexon_chain", addlN=9999, file=remaining_file)

removed_transcripts_file = save_basename + ".artificial_removed.gtf"
if os.path.exists(removed_transcripts_file):
    print(removed_transcripts_file, "exists. Removing it")
    os.remove(removed_transcripts_file)
for gid, chains in removed_transcripts:
    gchains = [chains] # put transcripts in a gene
    info = dict()
    tsm.save_novel_gene_in_gtf(gchains, info, gid, "pexon_chain", addlN=1, file=removed_transcripts_file)

Afterwards, we can test on those files as we did in the `long_read_support` module. By using `remaining_file` as input, and `removed_transcripts_file` as the ground truth.